<a href="https://colab.research.google.com/github/ikoojos/Algorithm-Debt-Research/blob/master/Random_Forest_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
from itertools import product
from sklearn.ensemble import RandomForestClassifier
import importlib

drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/AD Final Experiments')

from preprocessing import preprocess_data
from splitting import split_data
from utils import *
from evaluate_model import evaluate_best_model
from rf_tuning import hyperparameter_tuning_rf


for module in ['preprocessing', 'splitting', 'utils', 'evaluate_model', 'rf_tuning']:
    importlib.reload(sys.modules[module])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2]
}

vectorizer = CountVectorizer()
best_model, best_params, best_score = hyperparameter_tuning_rf(X_train_final, y_train_final, X_val, y_val, param_grid, vectorizer)
evaluate_best_model(best_model, best_params, best_score, X_test, y_test)

Best parameters found: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1}
Validation set accuracy: 0.8832797427652733

Test Confusion Matrix:
[[  70    0    2   89    1    7    0   31]
 [   0   38    0   34    0    3    4   10]
 [   0    1   57   53    0    5    5   14]
 [   2    4    9 1958    0   80   11  142]
 [   0    0    0    6   11    5    0    1]
 [   0    0    5  110    1  255    3   13]
 [   0    0    2   34    0    2   91   14]
 [   4    1    1  119    1    0    2 4464]]

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.92      0.35      0.51       200
         COMPATIBILITY       0.86      0.43      0.57        89
                DEFECT       0.75      0.42      0.54       135
                DESIGN       0.81      0.89      0.85      2206
         DOCUMENTATION       0.79      0.48      0.59        23
        IMPLEMENTATION       0.71      0.66      0.69       3